# [공공데이터 포털 - 기상청_단기예보 ((구)_동네예보) 조회서비스 Open API]를 이용하여 날씨 안내 AI 고객센터 챗봇 만들기

## Reference : https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15084084

# 초단기실황조회 API (getUltraSrtNcst) 사용법 살펴보기

In [ ]:
decoding_일반_인증키 = '여러분의_Decoding_일반_인증키'

In [ ]:
from datetime import datetime

def get_current_date():
    current_date = datetime.now().date()
    return current_date.strftime("%Y%m%d")

def get_current_hour():
    now = datetime.now()
    return datetime.now().strftime("%H%M")

In [ ]:
import requests

url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst' # 초단기실황조회
params = {
    'serviceKey' : decoding_일반_인증키,
    'pageNo' : '1',
    'numOfRows' : '1000',
    'dataType' : 'JSON',
    'base_date' : get_current_date(),
    'base_time' : get_current_hour(),
    'nx' : '55',
    'ny' : '127'
    }

response = requests.get(url, params=params)

In [ ]:
import json
result = json.loads(response.text)
result

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL_SERVICE'},
  'body': {'dataType': 'JSON',
   'items': {'item': [{'baseDate': '20240624',
      'baseTime': '0700',
      'category': 'PTY',
      'nx': 55,
      'ny': 127,
      'obsrValue': '0'},
     {'baseDate': '20240624',
      'baseTime': '0700',
      'category': 'REH',
      'nx': 55,
      'ny': 127,
      'obsrValue': '95'},
     {'baseDate': '20240624',
      'baseTime': '0700',
      'category': 'RN1',
      'nx': 55,
      'ny': 127,
      'obsrValue': '0'},
     {'baseDate': '20240624',
      'baseTime': '0700',
      'category': 'T1H',
      'nx': 55,
      'ny': 127,
      'obsrValue': '21.9'},
     {'baseDate': '20240624',
      'baseTime': '0700',
      'category': 'UUU',
      'nx': 55,
      'ny': 127,
      'obsrValue': '0.6'},
     {'baseDate': '20240624',
      'baseTime': '0700',
      'category': 'VEC',
      'nx': 55,
      'ny': 127,
      'obsrValue': '211'},
     {'baseDate': '20240624',
   

In [ ]:
weather_enum = {
    "0": "맑음",
    "1": "비",
    "2": "비/눈",
    "3": "눈",
    "5": "빗방울",
    "6": "빗방울눈날림",
    "7": "눈날림"
}

In [ ]:
def parse_weather_infomation(result):
    for item in result['response']['body']['items']['item']:
        if item['category'] == 'PTY':
            weather = weather_enum[item['obsrValue']]
        if item['category'] == 'REH':
            습도 = item['obsrValue']
        if item['category'] == 'RN1':
            한시간강수량 = item['obsrValue']
        if item['category'] == 'T1H':
            기온 = item['obsrValue']

    return_string = ''
    return_string += f'날씨 : {weather}\n'
    return_string += f'습도 : {습도}\n'
    return_string += f'1시간강수량 : {한시간강수량}\n'
    return_string += f'기온 : {기온}\n'

    return return_string

In [ ]:
weather_info = parse_weather_infomation(result)
print(weather_info)

날씨 : 맑음
습도 : 95
1시간강수량 : 0
기온 : 21.9



# 하나의 함수로 묶기

In [ ]:
def get_weather_infomation(decoding_일반_인증키, x_position, y_position):
    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst' # 초단기실황조회
    params ={
              'serviceKey' : decoding_일반_인증키,
              'pageNo' : '1',
              'numOfRows' : '1000',
              'dataType' : 'JSON',
              'base_date' : get_current_date(),
              'base_time' : get_current_hour(),
              'nx' : x_position,
              'ny' : y_position
             }

    response = requests.get(url, params=params)
    weather_info = parse_weather_infomation(json.loads(response.text))

    return weather_info

In [ ]:
# 서울특별시
x_position = '60'
y_position = '127'

weather_info = get_weather_infomation(decoding_일반_인증키, x_position, y_position)
print(weather_info)

날씨 : 맑음
습도 : 83
1시간강수량 : 0
기온 : 23.6



# 예보지점 X 좌표, 예보지점 Y 좌표 얻기

*   참고문서 - [기상청41_단기예보 조회서비스_오픈API활용가이드_최종.zip] 파일 다운로드후 압축풀기

In [ ]:
import pandas as pd

In [ ]:
file_path = '/content/기상청41_단기예보 조회서비스_오픈API활용가이드_격자_위경도(20240101).xlsx'

In [ ]:
# 데이터 프레임으로 읽어오기
df = pd.read_excel(file_path)
df

,구분,행정구역코드,1단계,2단계,3단계,격자 X,격자 Y,경도(시),경도(분),경도(초),위도(시),위도(분),위도(초),경도(초/100),위도(초/100),위치업데이트
0,kor,1100000000,서울특별시,NaN,NaN,60,127,126,58,48.03,37,33,48.85,126.980008,37.563569,NaN
1,kor,1111000000,서울특별시,종로구,NaN,60,127,126,58,53.91,37,34,13.36,126.981642,37.570378,NaN
2,kor,1111051500,서울특별시,종로구,청운효자동,60,127,126,58,14.35,37,35,2.89,126.970652,37.584137,NaN
3,kor,1111053000,서울특별시,종로구,사직동,60,127,126,58,15.44,37,34,23.77,126.970956,37.573269,NaN
4,kor,1111054000,서울특별시,종로구,삼청동,60,127,126,59,2.32,37,34,56.73,126.983978,37.582425,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3825,kor,5183031000,강원특별자치도,양양군,서면,87,138,128,35,15.55,38,4,7.26,128.587653,38.068683,20230611.0
3826,kor,5183032000,강원특별자치도,양양군,손양면,88,138,128,38,35.76,38,3,28.07,128.643267,38.057797,20230611.0
3827,kor,5183033000,강원특별자치도,양양군,현북면,89,137,128,43,31.15,38,0,58.57,128.725319,38.016269,20230611.0
3828,kor,5183034000,강원특별자치도,양양군,현남면,90,136,128,45,41.79,37,57,59.07,128.761608,37.966408,20230611.0


In [ ]:
df.columns

Index(['구분', '행정구역코드', '1단계', '2단계', '3단계', '격자 X', '격자 Y', '경도(시)', '경도(분)',
       '경도(초)', '위도(시)', '위도(분)', '위도(초)', '경도(초/100)', '위도(초/100)', '위치업데이트'],
      dtype='object')

In [ ]:
# 예보지점 X 좌표, 예보지점 Y를 얻어오는 함수 정의
def get_grid_coordinates(df, level1, level2=None, level3=None):
    query = f"`1단계` == '{level1}'"
    if level2 is not None:
        query += f" and `2단계` == '{level2}'"
    if level3 is not None:
        query += f" and `3단계` == '{level3}'"

    result = df.query(query)
    if not result.empty:
        grid_x = result.iloc[0]['격자 X']
        grid_y = result.iloc[0]['격자 Y']
        return grid_x, grid_y
    else:
        return None, None

In [ ]:
# 함수 사용 예시
level1 = '서울특별시'
level2 = '종로구'
level3 = '청운효자동'

grid_x, grid_y = get_grid_coordinates(df, level1, level2, level3)
print(f"격자 X: {grid_x}, 격자 Y: {grid_y}")

격자 X: 60, 격자 Y: 127


In [ ]:
level1 = '서울특별시'

grid_x, grid_y = get_grid_coordinates(df, level1)
print(f"격자 X: {grid_x}, 격자 Y: {grid_y}")

격자 X: 60, 격자 Y: 127


In [ ]:
# '서울특별시 종로구 청운효자동' 현재 날씨 정보얻기
x_poisition = '60'
y_poisition = '127'

weather_info = get_weather_infomation(decoding_일반_인증키, x_poisition, y_poisition)
print(weather_info)

날씨 : 맑음
습도 : 83
1시간강수량 : 0
기온 : 23.6



# 날씨 안내 AI 고객센터 챗봇 만들기

In [ ]:
!pip install --upgrade --quiet langchain langchainhub langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00


### Decoding 키 환경변수 설정

In [ ]:
import os
os.environ["decoding_일반_인증키"] = '여러분의_Decoding_일반_인증키'

### LangSmith 설정

In [ ]:
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Weather API Chatbot - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "여러분의_LANGSMITH_API_KEY"

In [ ]:
unique_id

'26c13e84'

###  OpenAI API Key 설정

In [ ]:
OPENAI_KEY = "여러분의_OPENAI_API_KEY"

### LLM 설정

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", openai_api_key=OPENAI_KEY)

### Tool 정의

*   df를 전역변수에서 가져오도록 함수 로직 수정


In [ ]:
# 예보지점 X 좌표, 예보지점 Y를 얻어오는 함수 정의
def get_grid_coordinates(level1, level2=None, level3=None):
    query = f"`1단계` == '{level1}'"
    if level2 is not None:
        query += f" and `2단계` == '{level2}'"
    if level3 is not None:
        query += f" and `3단계` == '{level3}'"

    result = df.query(query)
    if not result.empty:
        grid_x = result.iloc[0]['격자 X']
        grid_y = result.iloc[0]['격자 Y']
        return grid_x, grid_y
    else:
        return None, None

In [ ]:
from langchain_core.tools import tool

@tool
def get_weather_infomation(level1: str, level2: str = None, level3: str = None) -> str:
    """Get weather information."""
    x_position, y_position = get_grid_coordinates(level1, level2, level3)

    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst' # 초단기실황조회
    params = {
              'serviceKey' : os.environ.get('decoding_일반_인증키'),
              'pageNo' : '1',
              'numOfRows' : '1000',
              'dataType' : 'JSON',
              'base_date' : get_current_date(),
              'base_time' : get_current_hour(),
              'nx' : x_position,
              'ny' : y_position
             }

    response = requests.get(url, params=params)
    weather_info = parse_weather_infomation(json.loads(response.text))

    return weather_info

In [ ]:
print(get_weather_infomation.name)
print(get_weather_infomation.description)
print(get_weather_infomation.args)

get_weather_infomation
Get weather information.
{'level1': {'title': 'Level1', 'type': 'string'}, 'level2': {'title': 'Level2', 'type': 'string'}, 'level3': {'title': 'Level3', 'type': 'string'}}


In [ ]:
get_weather_infomation.invoke({"level1": '서울특별시'})

'날씨 : 맑음\n습도 : 83\n1시간강수량 : 0\n기온 : 23.6\n'

### tool agent 설정

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent

In [ ]:
# Get the prompt to use - can be replaced with any prompt that includes variables "agent_scratchpad" and "input"!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [ ]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlacehold

In [ ]:
tools = [get_weather_infomation]

In [ ]:
# Construct the tool calling agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
msg_result = agent_executor.invoke(
    {
        "input": "서울특별시의 날씨를 알려줘"
    }
)
msg_result



> Entering new AgentExecutor chain...

Invoking: `get_weather_infomation` with `{'level1': '서울특별시'}`


날씨 : 맑음
습도 : 83
1시간강수량 : 0
기온 : 23.6
서울특별시의 현재 날씨는 맑으며, 습도는 83%이고 기온은 23.6도입니다. 1시간 강수량은 0입니다.

> Finished chain.


{'input': '서울특별시의 날씨를 알려줘',
 'output': '서울특별시의 현재 날씨는 맑으며, 습도는 83%이고 기온은 23.6도입니다. 1시간 강수량은 0입니다.'}

In [ ]:
msg_result['output']

'서울특별시의 현재 날씨는 맑으며, 습도는 83%이고 기온은 23.6도입니다. 1시간 강수량은 0입니다.'

In [ ]:
msg_result = agent_executor.invoke(
    {
        "input": "서울특별시에 현재 비가 오는지 알려줘"
    }
)
msg_result



> Entering new AgentExecutor chain...

Invoking: `get_weather_infomation` with `{'level1': '서울특별시'}`


날씨 : 맑음
습도 : 83
1시간강수량 : 0
기온 : 23.6
서울특별시의 현재 날씨는 맑고, 기온은 23.6도이며 습도는 83%입니다. 1시간 강수량은 0입니다. 현재 비가 오지 않고 있습니다.

> Finished chain.


{'input': '서울특별시에 현재 비가 오는지 알려줘',
 'output': '서울특별시의 현재 날씨는 맑고, 기온은 23.6도이며 습도는 83%입니다. 1시간 강수량은 0입니다. 현재 비가 오지 않고 있습니다.'}

In [ ]:
msg_result['output']

'서울특별시의 현재 날씨는 맑고, 기온은 23.6도이며 습도는 83%입니다. 1시간 강수량은 0입니다. 현재 비가 오지 않고 있습니다.'